# Second attempt at making a model for 'In the Areoplane Over the Sea'

`I did more research into word level text generation and found a really helpful blog post that gave me a good starting point for creating more word level text generators. This model out put more coherent sentences than my previous attempts.`

In [1]:
# based on the LSTM RNN walkthrough on machine learning mastery (machineleanringmastery.com)

In [21]:
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras import models
import pickle

In [22]:
# load text
file = open('nmh_lyrics.txt', 'r')
text = file.read()
file.close()

In [23]:
# clean text
text = text.replace('[Verse ', '[verse')
text = text.replace("'", '')
tokens = text.split()
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in tokens]
tokens = [token.lower() for token in tokens]

In [24]:
# saving tokens
length = 25 + 1
sequences = []
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)

In [25]:
# save sequence
data = '\n'.join(sequences)
file = open('clean_nmh_lyrics.txt', 'w')
file.write(data)
file.close()

In [26]:
# load lyrics
file = open('clean_nmh_lyrics.txt', 'r')
text = file.read()
text = text.replace(',', '')
file.close
lines = text.split('\n')

In [28]:
# tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1

In [29]:
# set up data
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y)
seq_length = X.shape[1]

In [126]:
# model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [127]:
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
2153/2153 [==============================] - 4s 2ms/step - loss: 6.3981 - acc: 0.0390
Epoch 2/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.8398 - acc: 0.0390
Epoch 3/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6748 - acc: 0.0418
Epoch 4/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6395 - acc: 0.0418
Epoch 5/100
2153/2153 [==============================] - 3s 2ms/step - loss: 5.6346 - acc: 0.0418
Epoch 6/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6318 - acc: 0.0376
Epoch 7/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6262 - acc: 0.0418
Epoch 8/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6240 - acc: 0.0418
Epoch 9/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6172 - acc: 0.0553
Epoch 10/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.5860 - acc: 0.0687
Epoch 11/100
2153/2

2153/2153 [==============================] - 4s 2ms/step - loss: 1.8918 - acc: 0.5151
Epoch 84/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.8535 - acc: 0.5300
Epoch 85/100
2153/2153 [==============================] - 4s 2ms/step - loss: 1.8230 - acc: 0.5411
Epoch 86/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7919 - acc: 0.5467
Epoch 87/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7905 - acc: 0.5444
Epoch 88/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7417 - acc: 0.5536
Epoch 89/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7127 - acc: 0.5662
Epoch 90/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.6812 - acc: 0.5764
Epoch 91/100
2153/2153 [==============================] - 4s 2ms/step - loss: 1.6578 - acc: 0.5787A: 2s - loss: 1.5698 
Epoch 92/100
2153/2153 [==============================] - 4s 2ms/step - loss: 1.6240 - acc: 0.5936
Ep

In [131]:
# save model
model.save('base_model_from_Machine_Learning_Mastery.h5')

In [32]:
reverse_dict = dict([(index, word) for word, index in tokenizer.word_index.items()])

with open('neutral_milk_hotel_word_to_index.pkl', 'wb') as f:
    pickle.dump(tokenizer.word_index, file=f)

with open('neutral_milk_hotel_index_to_word.pkl', 'wb') as f:
    pickle.dump(reverse_dict, file=f)

In [209]:
def generate_seq(model=model,
                 tokenizer=tokenizer,
                 seq_length=seq_length,
                 seed_text=lines[np.random.randint(0,len(lines))],
                 n_words=500):
    result = []
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        word = reverse_dict[yhat[0]]
        in_text = in_text + ' ' + word
        result.append(word)
    return ' '.join(result)

In [210]:
generate_seq()

'ashes left born and theyll need screams and falls apart chorus two your mouth to say ourselves at least enough to carry to carry on standing communist the mountain love the enemies with jesus enemies i sank take you you will live forever you tomatoes and radio wires and wont safe and retire on a spine to say hang for your hollow ways moving your mouth to pull out to have found and ugly bursting to carry to carry to say again moving a face i when all bridge do do do do do do do do do do do do do do do do do do do do do do i will know you buried you will be one you you her sister and mother and fivehundred families and twist we is around a place all bend and spitting wanting and could left where a dead dog as and sleep to tore to pull out to be myself again things to try but your mouth on your miracles aimed to say keys for two the point shout with know you will love you spitting did she will take off our place you fivehundred wait for the rest of your eyes of some holy enemy ill crush 